# <center>**THUẬT TOÁN NAÏVE BAYES**<center>

## **Câu 1: Toy example – Phân loại loài hoa Iris (UCI Iris Dataset)**  
## **Câu 2: Nhận dạng ký tự A-Z (UCI Letter Recognition Dataset)**

---

In [16]:
import numpy as np
import pandas as pd
import random
import math
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

---
# **I. CÂU 1: PHÂN LOẠI LOÀI HOA IRIS (Toy Example)**

### **Giới thiệu**
- **Tập dữ liệu:** UCI Iris (150 mẫu, 4 đặc trưng, 3 lớp: `Iris-setosa`, `Iris-versicolor`, `Iris-virginica`)
- **Mục tiêu:** Demo thuật toán Naive Bayes từ đầu (from scratch)
- **Dữ liệu:** https://archive.ics.uci.edu/ml/datasets/iris
- **Tham khảo:** https://alphacoder.xyz/naive-bayes

### **Hướng dẫn cài đặt & chạy demo**
```bash
# 1. Cài đặt môi trường
pip install numpy pandas jupyter

# 2. Chạy Jupyter
jupyter notebook

In [17]:
# Tách dữ liệu theo lớp
def separate_by_class(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        class_value = vector[-1]
        if class_value not in separated:
            separated[class_value] = []
        separated[class_value].append(vector)
    return separated

# Tính mean, var cho từng đặc trưng
def summarize_dataset(class_rows):
    summaries = [(np.mean(column), np.var(column, ddof=1) + 1e-4) for column in zip(*class_rows)]
    return summaries

# Tính thống kê cho từng lớp
def summarize_by_class(dataset):
    separated = separate_by_class(dataset)
    summaries = {}
    total_rows = len(dataset)
    for class_value, rows in separated.items():
        class_data = [row[:-1] for row in rows]  # loại nhãn
        summaries[class_value] = {
            'summary': summarize_dataset(class_data),
            'prior': len(rows) / total_rows
        }
    return summaries

# Xác suất Gaussian
def calculate_probability(x, mean, var):
    exponent = math.exp(-((x - mean) ** 2) / (2 * var))
    return (1 / math.sqrt(2 * math.pi * var)) * exponent

# Tính posterior cho một mẫu
def calculate_class_probabilities(summaries, row):
    probabilities = {}
    for class_value, class_summary in summaries.items():
        probabilities[class_value] = class_summary['prior']
        for i in range(len(class_summary['summary'])):
            mean, var = class_summary['summary'][i]
            probabilities[class_value] *= calculate_probability(row[i], mean, var)
    return probabilities

# Dự đoán lớp
def predict(summaries, row):
    probabilities = calculate_class_probabilities(summaries, row)
    best_label = max(probabilities, key=probabilities.get)
    return best_label, probabilities

In [18]:
def load_iris_dataset(file_name="iris.data", train_ratio=0.67):
    df = pd.read_csv(file_name, header=None)
    df.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
    dataset = df.values.tolist()
    for row in dataset:
        row[:4] = [float(x) for x in row[:4]]
    random.seed(42)
    random.shuffle(dataset)
    split_idx = int(len(dataset) * train_ratio)
    return dataset[:split_idx], dataset[split_idx:]

# Tải dữ liệu
training_set, test_set = load_iris_dataset()
print(f"Đã tải Iris: {len(training_set)} train, {len(test_set)} test")

Đã tải Iris: 100 train, 50 test


In [19]:
print("\nBắt đầu huấn luyện Naïve Bayes từ scratch trên Iris...\n")

# Huấn luyện: tính thống kê
summaries = summarize_by_class(training_set)

# Dự đoán
predictions = []
correct_examples = []
wrong_examples = []

for idx, test_row in enumerate(test_set):
    features = test_row[:-1]
    true_label = test_row[-1]
    pred_label, probs = predict(summaries, features)
    predictions.append(pred_label)
    
    # Lưu ví dụ
    if pred_label == true_label and len(correct_examples) < 3:
        correct_examples.append((true_label, pred_label, probs))
    elif pred_label != true_label and len(wrong_examples) < 3:
        wrong_examples.append((true_label, pred_label, probs, features))

# Tính độ chính xác
accuracy = accuracy_score([row[-1] for row in test_set], predictions) * 100
print(f"{'='*60}")
print(f"--- KẾT QUẢ NAÏVE BAYES (TỪ ĐẦU) ---")
print(f"Độ chính xác: {accuracy:.2f}%")
print(f"{'='*60}\n")

# In ví dụ đúng
print("*** VÍ DỤ DỰ ĐOÁN ĐÚNG (3 mẫu) ***")
for i, (true, pred, probs) in enumerate(correct_examples, 1):
    print(f"Ví dụ {i}: THỰC TẾ: {true} → DỰ ĐOÁN: {pred}")
    print(f"   Xác suất: { {k: f'{v:.2e}' for k,v in probs.items()} }")
    print("---")

# In ví dụ sai
if wrong_examples:
    print("\n*** VÍ DỤ DỰ ĐOÁN SAI (nếu có) ***")
    for i, (true, pred, probs, feat) in enumerate(wrong_examples, 1):
        print(f"Ví dụ {i}: THỰC TẾ: {true} → DỰ ĐOÁN: {pred}")
        print(f"   4 đặc trưng: {feat}")
        print(f"   Xác suất: { {k: f'{v:.2e}' for k,v in probs.items()} }")
        print("---")
else:
    print("\nKhông có mẫu nào dự đoán sai!")


Bắt đầu huấn luyện Naïve Bayes từ scratch trên Iris...

--- KẾT QUẢ NAÏVE BAYES (TỪ ĐẦU) ---
Độ chính xác: 96.00%

*** VÍ DỤ DỰ ĐOÁN ĐÚNG (3 mẫu) ***
Ví dụ 1: THỰC TẾ: Iris-virginica → DỰ ĐOÁN: Iris-virginica
   Xác suất: {'Iris-versicolor': '6.68e-05', 'Iris-virginica': '2.22e-02', 'Iris-setosa': '3.42e-220'}
---
Ví dụ 2: THỰC TẾ: Iris-setosa → DỰ ĐOÁN: Iris-setosa
   Xác suất: {'Iris-versicolor': '6.28e-21', 'Iris-virginica': '1.24e-23', 'Iris-setosa': '1.66e-03'}
---
Ví dụ 3: THỰC TẾ: Iris-versicolor → DỰ ĐOÁN: Iris-versicolor
   Xác suất: {'Iris-versicolor': '4.75e-02', 'Iris-virginica': '4.40e-04', 'Iris-setosa': '6.37e-144'}
---

*** VÍ DỤ DỰ ĐOÁN SAI (nếu có) ***
Ví dụ 1: THỰC TẾ: Iris-versicolor → DỰ ĐOÁN: Iris-virginica
   4 đặc trưng: [6.7, 3.0, 5.0, 1.7]
   Xác suất: {'Iris-versicolor': '3.54e-03', 'Iris-virginica': '7.83e-02', 'Iris-setosa': '5.11e-193'}
---
Ví dụ 2: THỰC TẾ: Iris-versicolor → DỰ ĐOÁN: Iris-virginica
   4 đặc trưng: [5.9, 3.2, 4.8, 1.8]
   Xác suất: {'Iris

---
# **II. CÂU 2: NHẬN DẠNG KÝ TỰ A-Z (UCI Letter Recognition)**

### **Giới thiệu**
- **Tập dữ liệu:** 20,000 mẫu, 26 lớp (A-Z), 16 đặc trưng
- **Phân chia:** 16,000 huấn luyện, 4,000 kiểm tra
- **Mục tiêu:** So sánh **GaussianNB** và **MultinomialNB** từ `sklearn`
- **Dữ liệu:** https://archive.ics.uci.edu/ml/datasets/letter+recognition

In [20]:
def load_letter_dataset(file_name="letter-recognition.data"):
    df = pd.read_csv(file_name, header=None)
    df.columns = ['letter'] + [f'feat{i}' for i in range(16)]
    print(f"Đã tải: {len(df)} mẫu, {df['letter'].nunique()} lớp (A-Z)")
    return df

letter_df = load_letter_dataset()
X = letter_df.iloc[:, 1:].values.astype(float)
y = letter_df.iloc[:, 0].values

Đã tải: 20000 mẫu, 26 lớp (A-Z)


In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=16000, test_size=4000, random_state=42, stratify=y
)
print(f"Chia dữ liệu: {len(X_train)} train, {len(X_test)} test")

Chia dữ liệu: 16000 train, 4000 test


In [22]:
print("\n" + "="*50)
print("SO SÁNH GAUSSIAN VS MULTINOMIAL NAÏVE BAYES")
print("="*50)

# GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_g = gnb.predict(X_test)
acc_g = accuracy_score(y_test, y_pred_g)

# MultinomialNB
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred_m = mnb.predict(X_test)
acc_m = accuracy_score(y_test, y_pred_m)

print(f"\nGaussianNB   → Accuracy: {acc_g:.4f}")
print(f"MultinomialNB → Accuracy: {acc_m:.4f}")

print("\n--- Classification Report (GaussianNB) ---")
print(classification_report(y_test, y_pred_g, digits=2))


SO SÁNH GAUSSIAN VS MULTINOMIAL NAÏVE BAYES

GaussianNB   → Accuracy: 0.6522
MultinomialNB → Accuracy: 0.5553

--- Classification Report (GaussianNB) ---
              precision    recall  f1-score   support

           A       0.85      0.87      0.86       158
           B       0.48      0.72      0.57       153
           C       0.80      0.82      0.81       147
           D       0.61      0.71      0.66       161
           E       0.61      0.38      0.47       154
           F       0.70      0.77      0.73       155
           G       0.59      0.55      0.57       155
           H       0.58      0.35      0.44       147
           I       0.52      0.71      0.60       151
           J       0.83      0.71      0.77       149
           K       0.49      0.49      0.49       148
           L       0.91      0.78      0.84       152
           M       0.69      0.88      0.77       158
           N       0.89      0.68      0.77       157
           O       0.48      0.74 

---
# <center>**END**</center>